### Previously called Untitled (2)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
performance  = pd.read_csv('train/performance_train.csv', index_col= False)
facturation  = pd.read_csv('train/facturation_train.csv', index_col= False)
paiements    = pd.read_csv('train/paiements_train.csv', index_col= False)
transactions = pd.read_csv('train/transactions_train.csv', index_col= False)

print(sum(performance['Default'])/len(performance))

0.19336134453781512


In [3]:
performance["PERIODID_MY"]= pd.to_datetime(performance["PERIODID_MY"]).dt.year

In [4]:
performance

,ID_CPTE,PERIODID_MY,Default
0,99690111,2015,0
1,57427180,2012,0
2,29617912,2015,0
3,61632809,2015,0
4,14117855,2013,0
5,23700394,2013,0
6,27881705,2012,0
7,46100731,2012,0
8,58512689,2014,0
9,24661392,2016,0


In [5]:
transactions.head()

,ID_CPTE,MERCHANT_CATEGORY_XCD,MERCHANT_CITY_NAME,MERCHANT_COUNTRY_XCD,DECISION_XCD,PRIOR_CREDIT_LIMIT_AMT,TRANSACTION_AMT,TRANSACTION_CATEGORY_XCD,TRANSACTION_DTTM,TRANSACTION_TYPE_XCD,SICGROUP
0,99690111,A,365767,DP,C,5927.0,52.53,E,2015-06-20 12:00:00,F,AN
1,99690111,L,2635650,DP,C,13343.0,28.35,B,2015-01-25 12:00:00,F,AN
2,99690111,L,2635650,DP,C,13343.0,0.00,A,2015-01-26 12:00:00,G,AN
3,99690111,J,680536,AF,C,9430.0,0.00,A,2015-03-25 08:00:00,G,AW
4,99690111,J,680536,AF,C,10600.0,0.00,A,2015-03-03 08:00:00,G,AW


In [6]:
#Get rid of BS features
transaction_dropped = transactions.drop(["MERCHANT_CITY_NAME","MERCHANT_CATEGORY_XCD","MERCHANT_COUNTRY_XCD", "TRANSACTION_DTTM"],1)

In [7]:
## add credit limit minus transaction amount and drop credit limit, transaction amount
cred_minus_transaction = transaction_dropped["PRIOR_CREDIT_LIMIT_AMT"].sub(transaction_dropped["TRANSACTION_AMT"])
transaction_dropped = transaction_dropped.drop(["PRIOR_CREDIT_LIMIT_AMT", "TRANSACTION_AMT"],1)
transaction_dropped['cred_minus_transaction'] = cred_minus_transaction

In [8]:
# drop cred_minus_transaction and query whether it is positive
transaction_dropped["cred_minus_transaction_net_positive"] = transaction_dropped["cred_minus_transaction"].ge(0)
transaction_dropped = transaction_dropped.drop(["cred_minus_transaction"],1)

In [9]:
def summarize_by_ID(dataframe):
    output = {}
    DECISION_keys = transaction_dropped["DECISION_XCD"].value_counts().keys()

    TRANSACTION_C_keys = set(transaction_dropped["TRANSACTION_CATEGORY_XCD"])
    TRANSACTION_T_keys = set(transaction_dropped["TRANSACTION_TYPE_XCD"])
    cmt = set(transaction_dropped["cred_minus_transaction_net_positive"])
    SICGROUP_keys = set(transaction_dropped["SICGROUP"])
    for i in set(transaction_dropped["ID_CPTE"]):        
        '''
        i is a client id
        e.g. i = 996990111
        
        subframe is data specific to client i
        '''
        subframe = transaction_dropped.loc[transaction_dropped["ID_CPTE"] == i]
        #query for DECISION_XCD
        DECISION_dict = {}
        for j in DECISION_keys:
            s = "DECISION_XCD_" + j
            try:
                DECISION_dict[s] = subframe["DECISION_XCD"].value_counts(normalize=True)[j]
            except:
                DECISION_dict[s] = 0
        #query for transaction_c
        TRANSACTION_C_dict = {}
        for j in TRANSACTION_C_keys:
            s = "TRANSACTION_C_" + j
            try:
                TRANSACTION_C_dict[s] = subframe["TRANSACTION_CATEGORY_XCD"].value_counts(normalize=True)[j]
            except:
                TRANSACTION_C_dict[s] = 0
        TRANSACTION_T_dict = {}
        #query for transaction_t    
        for j in TRANSACTION_T_keys:
            s = "TRANSACTION_T_" + j
            try:
                TRANSACTION_T_dict[s] = subframe["TRANSACTION_TYPE_XCD"].value_counts(normalize=True)[j]
            except:
                TRANSACTION_T_dict[s] = 0
        #query for SICGROUP
        SICGROUP_dict = {}

        for j in SICGROUP_keys:
            s = "SCIGROUP_" + j
            try:
                SICGROUP_dict[s] = subframe["SICGROUP"].value_counts(normalize=True)[j]
            except:
                SICGROUP_dict[s] = 0
        CMT_dict = {}
        for j in cmt:
            s = "cred_minus_transaction_net_positive" + str(j)
            try:
                CMT_dict[s] = subframe["cred_minus_transaction_net_positive"].value_counts(normalize=True)[j]
            except:
                CMT_dict[s] = 0
                
        output[i] = [DECISION_dict, TRANSACTION_C_dict, TRANSACTION_T_dict, SICGROUP_dict, CMT_dict]
    return output


In [10]:
output = summarize_by_ID(transaction_dropped)
convert = {}
s = pd.Series()
for i in output.keys():
    for k in output[i]:
        s= {**s,**k}
    convert[i] = pd.Series(s)
    

In [11]:
final = pd.DataFrame.from_dict(convert, orient='index')
final

,DECISION_XCD_A,DECISION_XCD_B,DECISION_XCD_C,SCIGROUP_AA,SCIGROUP_AB,SCIGROUP_AC,SCIGROUP_AD,SCIGROUP_AE,SCIGROUP_AF,SCIGROUP_AG,...,TRANSACTION_C_E,TRANSACTION_T_A,TRANSACTION_T_B,TRANSACTION_T_C,TRANSACTION_T_D,TRANSACTION_T_E,TRANSACTION_T_F,TRANSACTION_T_G,cred_minus_transaction_net_positiveFalse,cred_minus_transaction_net_positiveTrue
10034823,0.000000,0.005797,0.994203,0.060870,0.000000,0.000000,0.000000,0.014493,0.000000,0.005797,...,0.918841,0.000000,0.028986,0.063768,0.000000,0.000000,0.904348,0.002899,0.023188,0.976812
10069450,0.000000,0.000000,1.000000,0.176991,0.008850,0.000000,0.044248,0.053097,0.017699,0.017699,...,0.725664,0.000000,0.044248,0.017699,0.017699,0.000000,0.884956,0.035398,0.000000,1.000000
10081565,0.000000,0.120253,0.879747,0.164557,0.018987,0.000000,0.000000,0.025316,0.000000,0.025316,...,0.715190,0.000000,0.056962,0.006329,0.000000,0.000000,0.886076,0.050633,0.012658,0.987342
10083607,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.007937,...,0.880952,0.000000,0.007937,0.000000,0.063492,0.015873,0.912698,0.000000,0.007937,0.992063
10097162,0.000000,0.000000,1.000000,0.021277,0.024316,0.000000,0.009119,0.009119,0.000000,0.012158,...,0.428571,0.000000,0.173252,0.012158,0.006079,0.000000,0.790274,0.018237,0.009119,0.990881
10109897,0.000000,0.000000,1.000000,0.244681,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,...,0.585106,0.000000,0.000000,0.010638,0.000000,0.000000,0.861702,0.127660,0.000000,1.000000
10174022,0.000000,0.000000,1.000000,0.278481,0.000000,0.000000,0.025316,0.075949,0.000000,0.050633,...,0.506329,0.000000,0.000000,0.000000,0.000000,0.000000,0.924051,0.075949,0.000000,1.000000
10179949,0.000000,0.047170,0.952830,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003145,...,0.037736,0.000000,0.012579,0.003145,0.000000,0.405660,0.569182,0.009434,0.707547,0.292453
10195847,0.000000,0.000000,1.000000,0.027778,0.006944,0.000000,0.000000,0.000000,0.000000,0.020833,...,0.895833,0.000000,0.048611,0.000000,0.000000,0.000000,0.951389,0.000000,0.000000,1.000000
10237041,0.000000,0.000000,1.000000,0.099010,0.000000,0.000000,0.009901,0.079208,0.000000,0.079208,...,0.742574,0.000000,0.019802,0.009901,0.000000,0.000000,0.881188,0.089109,0.000000,1.000000


In [12]:
#drop transaction date

paiements_drop = paiements.drop(["TRANSACTION_DTTM"],1)
paiements_drop["PAYMENT_REVERSAL_XFLG"].value_counts()

Q    292218
N        27
Name: PAYMENT_REVERSAL_XFLG, dtype: int64

In [13]:
def summarize_by_ID_2(dataframe):
    output = {}
    PAYMENT_REVERSAL_XFLG_key =  dataframe["PAYMENT_REVERSAL_XFLG"].value_counts().keys()
    for i in dataframe["ID_CPTE"].value_counts().keys():        
        subframe = dataframe.loc[dataframe["ID_CPTE"] == i]
        TRANSACTION_SUM_dict = {}
        TRANSACTION_SUM_dict["TRANSACTION_AMT_sum"] = subframe["TRANSACTION_AMT"].sum()

        PAYMENT_REVERSAL_XFLG_dict = {}
        for j in PAYMENT_REVERSAL_XFLG_key:
            s = "PAYMENT_REVERSAL_XFLG_key_" + str(j)
            try:
                PAYMENT_REVERSAL_XFLG_dict[s] = subframe["PAYMENT_REVERSAL_XFLG"].value_counts(normalize=True)[j]
            except:
                PAYMENT_REVERSAL_XFLG_dict[s] = 0   
        output[i] = [TRANSACTION_SUM_dict,PAYMENT_REVERSAL_XFLG_dict]
    return output

In [14]:
output2 = summarize_by_ID_2(paiements_drop)
convert2 = {}
s2 = pd.Series()
for i in output2.keys():
    for k2 in output2[i]:
        s2= {**s2,**k2}
    convert2[i] = pd.Series(s2)    



In [15]:
final2 = pd.DataFrame.from_dict(convert2, orient='index')


In [16]:
final.combine_first(final2)

,DECISION_XCD_A,DECISION_XCD_B,DECISION_XCD_C,PAYMENT_REVERSAL_XFLG_key_N,PAYMENT_REVERSAL_XFLG_key_Q,SCIGROUP_AA,SCIGROUP_AB,SCIGROUP_AC,SCIGROUP_AD,SCIGROUP_AE,...,TRANSACTION_C_E,TRANSACTION_T_A,TRANSACTION_T_B,TRANSACTION_T_C,TRANSACTION_T_D,TRANSACTION_T_E,TRANSACTION_T_F,TRANSACTION_T_G,cred_minus_transaction_net_positiveFalse,cred_minus_transaction_net_positiveTrue
10001822,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10007972,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10012520,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10025534,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10033579,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10034823,0.000000,0.005797,0.994203,0.0,1.0,0.060870,0.000000,0.000000,0.000000,0.014493,...,0.918841,0.0,0.028986,0.063768,0.000000,0.000000,0.904348,0.002899,0.023188,0.976812
10036020,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10068805,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10069450,0.000000,0.000000,1.000000,0.0,1.0,0.176991,0.008850,0.000000,0.044248,0.053097,...,0.725664,0.0,0.044248,0.017699,0.017699,0.000000,0.884956,0.035398,0.000000,1.000000
10081565,0.000000,0.120253,0.879747,0.0,1.0,0.164557,0.018987,0.000000,0.000000,0.025316,...,0.715190,0.0,0.056962,0.006329,0.000000,0.000000,0.886076,0.050633,0.012658,0.987342


In [17]:
print('performance length:', len(performance))
print('facturation length:', len(facturation))
print('paiements length:', len(paiements))
print('transactions length:', len(transactions))

performance length: 11900
facturation length: 166543
paiements length: 292320
transactions length: 690730


In [19]:
class customer:
    
    def __init__(self, default, assessment, facturation, paiements, transactions):
        
        self.default      = default
        self.assessment   = assessment
        self.facturation  = facturation
        self.paiements    = paiements
        self.transactions = transactions

In [31]:
len(set(facturation[facturation['DelqCycle'] > 0]['ID_CPTE']).intersection(set(performance[performance['Default'] == 1]['ID_CPTE'])))/3769

0.3982488723799416